In [82]:
from joblib import load
import logging
from pathlib import Path
import os
import pandas as pd
import locale
locale.setlocale(locale.LC_ALL, '')

'C/UTF-8/C/C/C/C'

In [96]:
class Predictor:
    def __init__(self, model_path, preprocessor_path):
        self.logger = logging.getLogger(__name__)
        try:
            self.model = load(model_path)
            self.preprocessor = load(preprocessor_path)
        except Exception as e:
            self.logger.error("Failed to load model or preprocessor: {}".format(e))
            raise e
    
    def parse_sample(self, sample):
        keys = ['Product Name', 'Form Factor', 'Processor Brand', 'Processor Speed',
                'Processor Count', 'RAM Size', 'Memory Clock Speed', 'Hard Drive Size',
                'Operating System', 'GraphicsCardRAM', 'Number of HDMI Ports',
                'BatteryLife', 'Display Type', 'is_SSD', 'HDMI',
                'Thunderbolt', 'DedicatedGraphics', 'IntegratedGraphics', 'Fingerprint',
                'BacklitKeyboard', 'RGBKeyboard', 'SoftwareIncluded', 'AdditionalInput',
                'USB_Ports', 'ResolutionType']
        
        return pd.DataFrame([dict(zip(keys, sample))])
    
    def format_prediction(self, prediction):
        return f'{int(round(prediction[0],2)*1000):,}'
    
    def predict(self, sample):
        try:
            sample = self.parse_sample(sample)
            sample = self.preprocessor.transform(sample)
            prediction = self.model.predict(sample)
            return prediction
        except Exception as e:
            self.logger.error("Failed to predict: {}".format(e))
            raise e
    

In [97]:
prd = Predictor(Path(os.getcwd()).parent / "models" / "random_forest_regressor_model.joblib", Path(os.getcwd()).parent / "models" / "preprocessor.joblib")

In [98]:
sample = ["""Lenovo IdeaPad Flex 5 Intel Core i5-1235U 14"(35.56cm) WUXGA IPS 2-in-1 Laptop (8GB/512GB SSD/Win 11/Office 2021/Backlit KB/Fingerprint/FHD Camera/Alexa/3 Month Game Pass/Storm Grey/1.5Kg),82R70068IN""",
          "thin and light",
          "Intel",
          1.3,
          10,
          8,
          2666,
          512,
          'Windows',
          0,
          1,
          8,
          'LCD',
          1,
          1,
          0,
          0,
          1,
          0,
          1,
          0,
          1,
          1,
          3,
          'FHD']
prd.format_prediction(prd.predict(sample))

'61,350'